# Import Python libraries

In [17]:
import xlwings as xw
import pandas as pd
import numpy as np
from poes.model.utils import param_poes

In [18]:
xb = xw.Book('poes/controller/control.xlsm')

In [19]:
sheet = xb.sheets['Resumen']

In [20]:
df_poes = sheet['A2'].options(pd.DataFrame, index=False, expand='table').value
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,None,None,None,None,None,None
1,Espesor,40.00,None,None,None,None,None,None
2,Porosida,0.16,None,None,None,None,None,None
3,Swi,0.51,None,None,None,None,None,None
4,Boi,2.60,None,None,None,None,None,None


# Exploratory data analysis

In [21]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min. ',
       'Lim. Max.'],
      dtype='object')

In [22]:
df_poes.columns = df_poes.columns.str.strip()

In [23]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [24]:
df_poes['Dist.'] = ['Triangular', 'Triangular', 'Log Normal', 'Normal', 'Exponencial']
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,None,None,None,None,None
1,Espesor,40.00,Triangular,None,None,None,None,None
2,Porosida,0.16,Log Normal,None,None,None,None,None
3,Swi,0.51,Normal,None,None,None,None,None
4,Boi,2.60,Exponencial,None,None,None,None,None


In [25]:
df_poes.shape

(5, 8)

In [26]:
df_poes['Loc'] = np.array([400, 0, 0, 0.4, 1])
df_poes['Scale'] = np.array([600, 90, 0.2, 0.2, 0.2])
df_poes['Sc'] = np.array([0.3, 0.3, 0.8, 0, 0.7])
df_poes['Lim. Min.'] = np.array([50, 0, 0, 0, 1])
df_poes['Lim. Max.'] = np.array([0, 180, 0.4, 1, 2])
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


# Use Loc method from pandas to extract specific values

In [27]:
df_poes.loc[2, "Data values"]

0.16

In [28]:
df_poes.loc[3, "Variables"]

'Swi'

# Call param_poes function

In [29]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [30]:
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


In [31]:
param_poes(df_poes, 1, 'Dist.', 'Loc', 'Scale', 100, 'Sc', 'Lim. Min.', 'Lim. Max.')

array([20.58661846, 69.44816307, 36.44813367, 23.03801475, 58.40103861,
       12.01422445, 28.53799651, 40.94900755, 55.25748146, 45.93859079,
       24.91889272, 27.37917292, 11.91093528, 84.20404269, 77.68506363,
       27.94532095, 26.61142001, 24.31934882, 58.62506386,  1.6413338 ,
       49.35902669, 21.67656382, 14.3731305 ,  8.79340894, 35.30921636,
       11.65460022, 42.59606573, 33.78529998, 54.10407341, 28.33117679,
        7.30023842, 35.30996197, 25.50765786, 31.21816796, 51.83960969,
       25.00641352, 55.68010165, 38.53246904, 31.13850816, 34.89781216,
       19.93191101, 32.44110965, 20.25723409, 68.0225155 , 30.29772576,
       15.69659213, 23.03627843, 30.18925257, 27.81770835, 30.06840168,
       87.44110661,  9.79056273, 16.27342273, 23.77497462, 13.87217597,
       48.76765062, 51.73433657, 39.66487974,  5.49195635, 12.50360826,
       38.20787824, 39.4203311 , 11.94936517, 72.45441811, 27.47446838,
       34.30045788, 43.8176396 , 44.73540249, 22.48053252, 45.70